# End-to-End

## 1. Problem (mini project)

Mini Project: FIFA - MoneyBall

Perform an end-to-end analysis putting into practice what you have learned so far. You will apply statistical or machine learning techniques and present your results to the class.



In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols

import warnings
warnings.filterwarnings('ignore')

## 2. Getting Data

In [2]:
data = pd.read_csv('fifa21_male2.csv')
data.head()

,ID,Name,Age,OVA,Nationality,Club,BOV,BP,Position,Player Photo,Club Logo,Flag Photo,POT,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,Gender
0,2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",NaN,€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
1,16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",NaN,€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
2,27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",NaN,€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
3,36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
4,41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",NaN,€5.5M,€12K,€7.2M,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male


In [3]:
data.shape

(17125, 107)

In [4]:
data.columns

Index(['ID', 'Name', 'Age', 'OVA', 'Nationality', 'Club', 'BOV', 'BP',
       'Position', 'Player Photo',
       ...
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK', 'Gender'],
      dtype='object', length=107)

#### A lot of meanings of the columns can be found in https://sofifa.com/

#### Some abbreviations:
OVA = overall scores
<br>BOV = best overall
<br>POT = potential scores
<br>GK = goalkeeping
<br>W/F = weak foot
<br>SM = skilled moves
<br>IR = international reputation
<br>CF = center forward

## 3. Cleaning/Wrangling/EDA

In [5]:
# standarize name of columns and remove ID column
data.columns = [col_name.lower().replace(' ', '_') for col_name in data.columns]
data = data.drop(data.columns[0], axis=1)
data.head()

,name,age,ova,nationality,club,bov,bp,position,player_photo,club_logo,flag_photo,pot,team_&_contract,height,weight,foot,growth,joined,loan_date_end,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,gender
0,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",NaN,€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
1,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",NaN,€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
2,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",NaN,€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
3,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
4,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",NaN,€5.5M,€12K,€7.2M,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male


#### Some more details about the dataset: 

"attacking" = sum ("crossing","finishing","heading_accuracy","short_passing","volleys")
<br>"skill" = sum ("dribbling","curve","fk_accuracy","long_passing","ball_control")
<br>"movement" = sum ("acceleration","sprint_speed","agility","reactions","balance")
<br>"power" = sum ("shot_power","jumping","stamina","strength","long_shots")
<br>"mentality" = sum ("aggression","interceptions","positioning","vision","penalties","composure")
<br>"defending" = sum ("marking","standing_tackle","sliding_tackle")
<br>"goalkeeping" = sum ("gk_diving","gk_handling","gk_kicking","gk_positioning","gk_reflexes")
<br>"total_stats" = sum ("attacking","skill","movement","power","mentality","defending","goalkiping")
<br>"base_stats" = sum ("pac","sho","pas","dri","def","phy")

In [6]:
# columns to drop
drop_columns = ["position","player_photo","club_logo","flag_photo","team_&_contract","joined","contract",
                "crossing","finishing","heading_accuracy","short_passing","volleys",
                "dribbling","curve","fk_accuracy","long_passing","ball_control",
                "acceleration","sprint_speed","agility","reactions","balance",
                "shot_power","jumping","stamina","strength","long_shots",
                "aggression","interceptions","positioning","vision","penalties","composure",
                "marking","standing_tackle","sliding_tackle",
                "gk_diving","gk_handling","gk_kicking","gk_positioning","gk_reflexes",
                "total_stats","base_stats","pac","sho","pas","dri","def","phy",
                "ls","st","rs","lw","lf","cf","rf","rw","lam","cam","ram","lm","lcm",
                "cm","rcm","rm","lwb","ldm","cdm","rdm","rwb","lb","lcb","cb","rcb","rb","gk","gender"]

In [7]:
data = data.drop(drop_columns, axis=1)

In [8]:
# rename the remaining columns with their corresponding meanings
data = data.rename(columns={"ova":"overall_scores",
                        "bov":"best_overall",
                        "pot":"potential_scores",
                        "w/f":"weak_foot",
                        "ir":"international_reputation",
                        "gk":"goalkeeper",
                        "cf":"center_forward",
                        "lde":"loan_date_end",
                        "bp":"better_position", 
                        "sm":"skilled_moves",
                        "a/w":"attacking_work_rate",
                        "d/w":"defensive_work_rate",})
data.head()

,name,age,overall_scores,nationality,club,best_overall,better_position,potential_scores,height,weight,foot,growth,loan_date_end,value,wage,release_clause,attacking,skill,movement,power,mentality,defending,goalkeeping,weak_foot,skilled_moves,attacking_work_rate,defensive_work_rate,international_reputation,hits
0,G. Pasquale,33,69,Italy,Udinese,71,LWB,69,"6'0""",181lbs,Left,0,NaN,€625K,€7K,€0,313,338,347,347,320,208,56,3 ★,2★,Medium,High,2 ★,4
1,Luis García,37,71,Spain,KAS Eupen,70,CM,71,"5'10""",143lbs,Right,0,NaN,€600K,€7K,€1.1M,337,369,305,324,362,153,56,4 ★,3★,Medium,Medium,1 ★,4
2,J. Cole,33,71,England,Coventry City,71,CAM,71,"5'9""",161lbs,Right,0,NaN,€1.1M,€15K,€0,337,387,295,284,317,99,51,4 ★,4★,Medium,Low,2 ★,11
3,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,82,"5'11""",165lbs,Right,14,NaN,€0,€0,€0,264,255,176,239,271,75,68,3 ★,1★,NaN,NaN,1 ★,3
4,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,81,"5'7""",150lbs,Right,0,NaN,€5.5M,€12K,€7.2M,367,408,346,297,370,181,45,4 ★,4★,High,Medium,4 ★,149


In [9]:
# NAN values
data.isna().sum()

name                            0
age                             0
overall_scores                  0
nationality                     0
club                           23
best_overall                    0
better_position                 0
potential_scores                0
height                          0
weight                          0
foot                            0
growth                          0
loan_date_end               16215
value                           0
wage                            0
release_clause                  0
attacking                       0
skill                           0
movement                        0
power                           0
mentality                       0
defending                       0
goalkeeping                     0
weak_foot                       0
skilled_moves                   0
attacking_work_rate            89
defensive_work_rate            89
international_reputation        0
hits                            0
dtype: int64

In [10]:
# column `loan_date_end` has a large number of NaN values (16,215) --> drop
data = data.drop(["loan_date_end"], axis=1)

In [11]:
data.isna().sum()

name                         0
age                          0
overall_scores               0
nationality                  0
club                        23
best_overall                 0
better_position              0
potential_scores             0
height                       0
weight                       0
foot                         0
growth                       0
value                        0
wage                         0
release_clause               0
attacking                    0
skill                        0
movement                     0
power                        0
mentality                    0
defending                    0
goalkeeping                  0
weak_foot                    0
skilled_moves                0
attacking_work_rate         89
defensive_work_rate         89
international_reputation     0
hits                         0
dtype: int64

In [12]:
# fill NAN with "unknown" in the column club since it is important
data["club"] = data["club"].fillna("unknown")
data.shape

(17125, 28)

In [13]:
# drop the remaining NAN values
data = data.dropna(axis=0)
data.shape

(17036, 28)

In [14]:
data.dtypes

name                        object
age                          int64
overall_scores               int64
nationality                 object
club                        object
best_overall                 int64
better_position             object
potential_scores             int64
height                      object
weight                      object
foot                        object
growth                       int64
value                       object
wage                        object
release_clause              object
attacking                    int64
skill                        int64
movement                     int64
power                        int64
mentality                    int64
defending                    int64
goalkeeping                  int64
weak_foot                   object
skilled_moves               object
attacking_work_rate         object
defensive_work_rate         object
international_reputation    object
hits                        object
dtype: object

#### Columns that are objects and should be numerical:
height, weight, value, wage, release_clause, weak_foot, skilled_moves, international_reputation, hits

In [15]:
# height: convert to cm
data["height"].value_counts()

6'0"     2572
5'10"    2406
5'9"     2015
5'11"    1931
6'2"     1860
6'1"     1799
5'8"      955
6'3"      951
5'7"      892
6'4"      714
5'6"      306
6'5"      254
5'5"      184
6'6"      110
5'4"       28
6'7"       27
5'3"       12
6'8"       11
5'2"        5
6'9"        2
5'1"        2
Name: height, dtype: int64

In [16]:
data["height"] = data["height"].str.replace('"','').apply(lambda x: round(int(x.split("'")[0])*30.48 + int(x.split("'")[1])*2.54))
data["height"].unique()

array([183, 178, 175, 170, 173, 180, 193, 168, 188, 190, 198, 185, 165,
       196, 201, 157, 203, 163, 160, 206, 155], dtype=int64)

In [17]:
# weight: convert to kg
data["weight"] = data["weight"].str.replace('lbs','').apply(lambda x: round(int(x)*0.45359237))
data["weight"].unique()

array([ 82,  65,  73,  68,  74,  72,  71,  76,  75,  95,  91,  70,  79,
        92,  80,  83,  69,  90,  93,  84,  77,  78,  64,  88,  85,  89,
        86,  81,  67,  66, 100,  87, 110,  61,  94,  96,  63, 101, 103,
        99,  60,  58,  62,  97,  59,  57,  98, 107,  56, 102, 104,  55,
        53,  50,  54,  52], dtype=int64)

In [18]:
# value
def clean_value(x):
    x = x.replace("€","")
    if "M" in x:
        x = float(x.replace("M","")) * 1000000
    elif "K" in x:
        x = float(x.replace("K","")) * 1000
    else:
        x = float(x)
    x = int(x)
    return x

data["value"] = list(map(clean_value, data["value"])) 
data["value"].unique()

array([   625000,    600000,   1100000,   5500000,    725000,   2800000,
            1000,    400000,    120000,         0,    500000,    190000,
         2200000,    350000,   1300000,   1500000,    275000,    325000,
          900000,     80000,   3000000,     35000,    975000,    425000,
          850000,     45000,   3200000,   1200000,     50000,    825000,
          130000,   1800000,    575000,   2100000,   1600000,    375000,
         2700000,    550000,   1700000,   3400000,    150000,    300000,
         4200000,   5000000,   1900000,   6500000,    875000,  15500000,
          210000,    450000,   1000000,    775000,   1400000,    100000,
          240000,    650000,    950000,   6000000,   3900000,  46000000,
           30000,   2500000,   4099999,     15000,   7500000,   4300000,
           25000,   9500000,   3100000,     90000,   2400000,    675000,
          475000,   2300000,    925000,   4700000,   4500000,    230000,
         2000000,     70000,   7000000,    170000, 

In [19]:
# wage: same as value
data["wage"] = list(map(clean_value, data["wage"]))
data["wage"].unique()

array([  7000,  15000,  12000,   5000,  44000,  60000,  40000,      0,
        10000,  30000,  45000,   6000,  41000,   3000,  20000,   4000,
         2000,  35000,   8000,  25000, 160000,  80000,   9000,  70000,
       100000,  11000, 220000,  90000,    500,  19000,   1000,  50000,
        49000,  22000,  21000,  33000,    750, 110000,  23000,  16000,
          700,  13000,  17000,  31000,    600,    550,  27000,  14000,
       200000,  28000,  37000,  94000,  32000,  29000,  18000,    650,
        34000,  53000,  47000,  43000,  26000,  36000,  79000,  24000,
       300000,  66000,  62000,  54000,  65000, 560000,    900,  42000,
        57000,  93000,  48000,  51000, 350000,  81000, 125000,  58000,
        39000,  85000,  98000,  38000,  46000,  76000,  84000, 130000,
       115000,  83000, 170000,    800, 260000,  59000,  72000,    950,
        55000,  61000,  56000,  96000, 150000,  63000,  87000,    850,
       140000,  64000, 310000,  89000,  82000, 120000, 105000,  68000,
      

In [20]:
# release_clause: same as value
data["release_clause"] = list(map(clean_value, data["release_clause"]))
data["release_clause"].unique()

array([      0, 1100000, 7200000, ...,  121000,  142000,  367000],
      dtype=int64)

In [21]:
# weak_foot
data["weak_foot"] = data["weak_foot"].str.replace(' ★','').str.strip().astype(int)
data["weak_foot"].unique()

array([3, 4, 2, 5, 1])

In [22]:
# skilled_moves
data["skilled_moves"] = data["skilled_moves"].str.replace('★','').str.strip().astype(int)
data["skilled_moves"].unique()

array([2, 3, 4, 1, 5])

In [23]:
# international_reputation
data["international_reputation"] = data["international_reputation"].str.replace(' ★','').str.strip().astype(int)
data["international_reputation"].unique()

array([2, 1, 4, 3, 5])

In [24]:
data["hits"] = list(map(lambda x: int(float(str(x).replace("K", "")) * 1000) if "K" in str(x) else int(x), data["hits"]))
data["hits"].unique()

array([   4,   11,  149,    7,    9,   16,   10,    3,    5,    8,  154,
          6,   18,   22,   34,   19,   15,   14,   27,   21,  309,   32,
        151,  750,   13,   39,   83,   20,  442,   92,   23,   29,   30,
         12,   31,  297,   36,   90,   88,   24,  171,   40,   25,   60,
        103,   38,   82,   57,   17,   26,   44,   67,  571,   61,   68,
         37,  231,   58,   62,   28,   51,   45,   80,  162,   59,  138,
        220,   93,   47,  281,   33,   46,   35,  830,   43,   64,   49,
         48,   54,  259,  161,   74,  434,   42,  131,  222,  229,  125,
        142,   81,  128,   78,  109,   72,   75,  111,   63,  123,  136,
         89,   55,  663,   97,   41,  143,   73,  464,  232,  164,  212,
        129,   79,  107,   69,  117,   71,  101,  202,  140,  110,   66,
       6300,  122,   95,  189,  157,  215,   52,  248,  178,  238,  208,
        214,  407, 1200,  349,  180,  224,  155,  147,   98,   99,  152,
        148,  120,  167,  158,   70,  328,  183,   

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17036 entries, 0 to 17124
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   name                      17036 non-null  object
 1   age                       17036 non-null  int64 
 2   overall_scores            17036 non-null  int64 
 3   nationality               17036 non-null  object
 4   club                      17036 non-null  object
 5   best_overall              17036 non-null  int64 
 6   better_position           17036 non-null  object
 7   potential_scores          17036 non-null  int64 
 8   height                    17036 non-null  int64 
 9   weight                    17036 non-null  int64 
 10  foot                      17036 non-null  object
 11  growth                    17036 non-null  int64 
 12  value                     17036 non-null  int64 
 13  wage                      17036 non-null  int64 
 14  release_clause        

## 4. Processing Data

## 5. Modeling

### Linear Regression

## 6. Model Validation

## 7. Reporting